# A.I. Lab - 3

### Heuristic Function

cost_function = Manhattan Distance

In [ ]:
def generate_heuristic(self):
        self.heuristic=0
        for num in range(1,9):
            distance=abs(self.state.index(num) - self.goal_state.index(num))
            i=int(distance/3)
            j=int(distance%3)
            self.heuristic=self.heuristic+i+j

###ManhattanDistance

In [ ]:
def ManhattanDistance(state):
      goal_state = [1,2,3,
                  8,0,4,
                  7,6,5]
      heuristic = 0
      for i in range(len(goal_state)):
        ind = list(goal_state).index(state[i])
        if (ind != i):
          row_goal = int(i/3)
          row_ini = int(ind/3)
          col_goal = i%3
          col_ini = ind%3
          heuristic = heuristic + abs(row_goal - row_ini) + abs(col_goal - col_ini)
     #print(heuristic)

state = [0,1,3,
         8,2,4,
          7,6,5]
ManhattanDistance(state)                

4


### Combining it all 

In [ ]:
class Puzzle:
    goal_state = [1,2,3,
                  8,0,4,
                  7,6,5]
    heuristic=None #estimated cost
    g=None        #actual cost
    f=None # f=g=h total heuristic
    needs_hueristic=True
    num_of_instances=0
    def __init__(self,state,parent,action,g=0,needs_hueristic=True): ##g=0 will be for root  and,,, g=1 as every child created would be one step away form parent
        self.parent=parent
        self.state=state
        self.action=action
        if parent:
            self.g = parent.g + g
        else:
            self.g = g
        if needs_hueristic:
            self.needs_hueristic=True
            self.ManhattanDistance() # calculates estimated distance i.e self.heuristic
            self.f=self.heuristic+self.g  #adding totatl heuristic cost f=g+h
        Puzzle.num_of_instances+=1

    def __str__(self):
        return str(self.state[0:3])+'\n'+str(self.state[3:6])+'\n'+str(self.state[6:9])

    def generate_heuristic(self):
        self.heuristic=0
        for num in range(1,9):
            distance=abs(self.state.index(num) - self.goal_state.index(num))
            i=int(distance/3)
            j=int(distance%3)
            self.heuristic=self.heuristic+i+j

    def ManhattanDistance(self):
      self.heuristic = 0
      for i in range(len(self.goal_state)):
        ind = list(self.goal_state).index(self.state[i])
        if (ind != i):
          row_goal = int(i/3)
          row_ini = int(ind/3)
          col_goal = i%3
          col_ini = ind%3
          self.heuristic = self.heuristic + abs(row_goal - row_ini) + abs(col_goal - col_ini)
      

    def goal_test(self):
        if self.state == self.goal_state:
            return True
        return False

    @staticmethod
    def find_legal_actions(i,j):
        legal_action = ['U', 'D', 'L', 'R']
        if i == 0:  # up is disable
            legal_action.remove('U')
        elif i == 2:  # down is disable
            legal_action.remove('D')
        if j == 0:
            legal_action.remove('L')
        elif j == 2:
            legal_action.remove('R')
        return legal_action


    def MisplacedTiles(self):
      self.heuristic = 0
      for i in range(len(self.goal_state)):
        if self.goal_state[i] != self.state[i]:
          self.heuristic = self.heuristic + 1


    def generate_child(self):
        children=[]
        x = self.state.index(0)
        i = int(x / 3)
        j = int(x % 3)
        legal_actions=self.find_legal_actions(i,j)

        for action in legal_actions:
            new_state = self.state.copy()
            if action is 'U':
                new_state[x], new_state[x-3] = new_state[x-3], new_state[x]
            elif action is 'D':
                new_state[x], new_state[x+3] = new_state[x+3], new_state[x]
            elif action is 'L':
                new_state[x], new_state[x-1] = new_state[x-1], new_state[x]
            elif action is 'R':
                new_state[x], new_state[x+1] = new_state[x+1], new_state[x]            
            children.append(Puzzle(new_state,self,action,1,self.needs_hueristic)) ## passing g=1
        return children

    def find_solution(self):
        solution = []
        solution.append(self.action)
        path = self
        while path.parent != None:
            path = path.parent
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution

### Using above Class in Breadth First Search (ManhattanDistance)

In [ ]:
from queue import Queue

def breadth_first_search(initial_state):
    start_node = Puzzle(initial_state, None, None,0 )
    if start_node.goal_test():
        return start_node.find_solution()
    q = Queue()
    q.put(start_node)
    explored=[]
    while not(q.empty()):
        node=q.get()
        explored.append(node.state)
        children=node.generate_child()
        for child in children:
            if child.state not in explored:
                if child.goal_test():
                    return child.find_solution()
                q.put(child)
    return
state=[1, 3, 4, 
        8, 6, 2, 
        7, 0, 5]
breadth_first_search(state)

['U', 'R', 'U', 'L', 'D']

###Best First Search (Manhattan Distance)

In [ ]:
# best first search using stack
def best_first_search(initial_state):
    start_node = Puzzle(initial_state, None, None, 1)
    if start_node.goal_test():
        return start_node.find_solution()
    q=[] 
    
    q.append(start_node) 
    explored=[]
    while not(len(q)<=0):
        node=q.pop()
        explored.append(node.state)
        children=node.generate_child()
        for child in children:
            if child.state not in explored:
                if child.goal_test():
                    return child.find_solution()

      # arranging children in descending order so that small value of heuristic is appended at last
     #as  stack is implemented and will pick last value first
        for i in range(len(children)):
          for j in range(i,len(children)):
            if children[i].heuristic < children[j].heuristic:
              temp  = children[i]
              children[i] = children[j]
              children[j] = temp
        bestchild=children.pop() #getting and adding the last child as it is the smallest in stack
        if bestchild.state not in explored:
            q.append(child)
            
           
            
    return False
# state=[0,1,3,
#                   8,2,4,
#                   7,6,5]

# best_first_search(state)

###DFS

In [ ]:
def Depth_first_search(initial_state):
  start_node= Puzzle(initial_state, None, None, 0)
  if start_node.goal_test():
    return start_node.find_solution()
  q=[] 
  q.append(start_node) 
  explored=[]
  while (len(q)>0):
    last_node = q.pop()
    explored.append(last_node.state) 
    children=last_node.generate_child()
    for child in children:
        if child.state not in explored:
            if child.goal_test():
                return child.find_solution()
            q.append(child)
  return False



####Sorting heuristic code check

In [ ]:
children=[2,1,4,3]
for i in range(len(children)):
          for j in range(len(children)):
            if children[i] < children[j]:
              temp  = children[i]
              children[i] = children[j]
              children[j] = temp
print(children)
for i in range(len(children)):
          for j in range(i,len(children)): 
           # print(i,children[i])
            #print(j,children[j])    # arranging children in descending order so that small value of heuristic is appended at last 
            if children[i] < children[j]: #as it is stack and will pick lasst value first
              temp  = children[i]
              children[i] = children[j]
              children[j] = temp
              
print(children)

[1, 2, 3, 4]
[4, 3, 2, 1]


####A star search

In [ ]:
def A_Star_search(initial_state):
    start_node = Puzzle(initial_state, None, None,0, 1)
    if start_node.goal_test():
        return start_node.find_solution()
    q=[] 
    q.append(start_node) # get the root node
    explored=[]
    while not(len(q)<=0):
        node=q[0] # pick the first node
        current_index=0
        for index, item in enumerate(q): # we pick the node with lowest heuristic
            if item.f < node.f:           #if f=g+h total heuristic cost of any item is less than currect selected node then get that node
                node = item
                current_index = index

        q.pop(current_index) # delete the selected node because we are going to visit it
        explored.append(node.state) # add it to explored/visited list
        if node.goal_test(): # check if this node is required answer 
              return node.find_solution()
  
        children=node.generate_child()
        for child in children:
            if child.state not in explored:
                if child.goal_test():
                    return child.find_solution()
                q.append(child)    
    return False
# state=[0,1,3,
#                    8,2,4,
#                    7,6,5]

# A_Star_search(state)

['R', 'D']

###GlobalbestFirst search

In [ ]:
from queue import Queue


def global_best_first_search(initial_state):
    start_node = Puzzle(initial_state, None, None, 1)
    if start_node.goal_test():
        return start_node.find_solution()
    q = Queue()
    q.put(start_node)
    explored=[]
    while not(q.empty()):
        node=q.get()
        
        explored.append(node.state)
        children=node.generate_child()
        for child in children:
            if child.state not in explored:
                if child.goal_test():
                    return child.find_solution()
      
        for i in range(len(children)):
          for j in range(len(children)):
            if children[i].heuristic > children[j].heuristic: #arrange in ascending order , cause using queue so need first value need to be small
              temp  = children[i]
              children[i] = children[j]
              children[j] = temp

        for child in children:
            if child.state not in explored:
                q.put(child)
    return


### Main Function

In [ ]:
from time import time
import operator
state=[
       [8,1,3,
        7,2,4,
        0,6,5],
       #[1, 3, 4, 
        #8, 6, 2, 
        #7, 0, 5],

       [2, 8, 1,
        0, 4, 3,
        7, 6, 5],

       [2, 8, 1,
        4, 6, 3,
        0, 7, 5]
      ]
      ####1) Created two dictionaries which stores the time and space complexity respectively, 
      ##### we will use these dictionaries to compare values of only comeplete 
      ####2) Only if solution is Comeplete we will update values in dictionaries
      ####3) Initial Values set to 0 so at the end which ever value is still zero will be removed from the dictionaries
timedic={"bfs_time":0, ##bfs->breadthFirst search
         "dfs_time":0,
         "bestfs_time":0,
         "GBfs_time":0,
         "asf_time":0,
        }
spacedic={"bfs_space":0,
         "dfs_space":0,
         "bestfs_space":0,
         "GBfs_space":0,
          "asf_space":0,
        }
print("We can see the Comparison in Following Table")
print("Breadth_FirstSearch:\t",end="")
for i in range(0,3):
    c=False
    Puzzle.num_of_instances=0
    t0 = time()
    bfs = breadth_first_search(state[i])
    t1 = time()
    
    if(bfs):
      print('Compelete  ',end ="")
      c=True
    else:
      print("InCompelete",end="")  
    print(' space:',Puzzle.num_of_instances,end =" ")
    if(c):
      timedic.update({"bfs_time": t1})
      
      spacedic.update({"bfs_space": Puzzle.num_of_instances})
    print('time:',t1,"\t| ",end =" ")



print("")
print("Best___FirstSearch :\t",end="")
for i in range(0,3):
    c=False
    Puzzle.num_of_instances=0
    t0 = time()
    bfs = best_first_search(state[i])
    t1 = time()-t0
   
    if(bfs):
      print('Compelete  ',end ="")
      c=True
    else:
      print("InCompelete",end="")  
    print(' space:',Puzzle.num_of_instances,end =" ")
    if(c):
      timedic.update({"bestfs_time": t1})
      
      spacedic.update({"bestfs_space": Puzzle.num_of_instances})
    print('time:',t1,"\t| ",end =" ")

print("")
print("Global_bestFSearch :\t",end="")
for i in range(0,3):
    c=False
    Puzzle.num_of_instances=0
    t0 = time()
    bfs = global_best_first_search(state[i])
    t1 = time()-t0
    
    if(bfs):
      print('Compelete  ',end ="")
      c=True
    else:
      print("InCompelete",end="")  
    print(' space:',Puzzle.num_of_instances,end =" ")
    if(c):
      timedic.update({"GBfs_time": t1})
      
      spacedic.update({"GBfs_space": Puzzle.num_of_instances})
    print('time:',t1,"\t| ",end =" ")

print("")
print("Depth_FirstSearch :\t",end="")
for i in range(0,1): ####### limited to one state or it takes too much time to give output
    c=False
    Puzzle.num_of_instances=0
    t0 = time()
    bfs = Depth_first_search(state[i])
    t1 = time()-t0
   
    if(bfs):
      print('Compelete  ',end ="") ####kindly, ignore the answer of dfs on total time and space on comparison
      c=True
    else:
      print("InCompelete",end="")  
    print(' space:',Puzzle.num_of_instances,end =" ")
    if(c):
      timedic.update({"dfs_time": t1})
      
      spacedic.update({"dfs_space": Puzzle.num_of_instances})
    print('time:',t1,"\t| ",end =" ")
print("")
############ A Starrr Search##############
print("A* Star search :\t",end="")
for i in range(0,3):
    c=False
    Puzzle.num_of_instances=0
    t0 = time()
    asf = A_Star_search(state[i])
    t1 = time()-t0 
    if(asf):
      print('Compelete  ',end ="")
      c=True
    else:
      print("InCompelete",end="")  
    print(' space:',Puzzle.num_of_instances,end =" ")
    if(c):
      timedic.update({"asf_time": t1}) 
      spacedic.update({"asf_space": Puzzle.num_of_instances})
    print('time:',t1,"\t| ",end =" ")
  
print("")
print("")
  #     ------------ seraches end here--------------
##
##      -
### Arranging with repect to time and space compelexity
###
##
listr=[]
for key in timedic: #removing time of incomepelte algos
  if(timedic[key]==0):
    listr.append(key)
for i in listr:
  timedic.pop(i)
print("*****************Arranging in order of best to worse total-time to solve all three problems of only comeplete solutions*******************")
sorted_t = sorted(timedic.items(), key=operator.itemgetter(1))

print(sorted_t)
print("")
print("")
listr=[]
for key in spacedic: ## removing space of incompelete 
  if(spacedic[key]==0):
    listr.append(key)
for i in listr:
  spacedic.pop(i)
print("#################Arranging in order of best to worse total-space to solve all three problems comeplete solutions#################")
sorted_s = sorted(spacedic.items(), key=operator.itemgetter(1))
print(sorted_s)

We can see the Comparison in Following Table
Breadth_FirstSearch:	Compelete   space: 24 time: 1648562722.1664817 	|  Compelete   space: 591 time: 1648562722.181205 	|  Compelete   space: 2956 time: 1648562722.2838488 	|  
Best___FirstSearch :	InCompelete space: 8 time: 0.00014281272888183594 	|  InCompelete space: 11 time: 0.00019240379333496094 	|  InCompelete space: 8 time: 0.0001404285430908203 	|  
Global_bestFSearch :	Compelete   space: 36 time: 0.0006940364837646484 	|  Compelete   space: 687 time: 0.019232988357543945 	|  Compelete   space: 3265 time: 0.21051859855651855 	|  
Depth_FirstSearch :	Compelete   space: 91 time: 0.001363992691040039 	|  
A* Star search :	Compelete   space: 11 time: 0.00018835067749023438 	|  Compelete   space: 50 time: 0.0006892681121826172 	|  Compelete   space: 79 time: 0.0010707378387451172 	|  

*****************Arranging in order of best to worse total-time to solve all three problems of only comeplete solutions*******************
[('asf_time', 0